# Shopify Data Science Internship Challenge

For a summary of the answers for question 1, scroll down to the bottom. In this notebook contains exploratory data analysis of sneaker sales from Shopify sneaker shops, as well as SQL queries on orders of food items. 

### Question 1: Who is buying these expensive shoes?

First we will do some exploratory data analysis using the database of Shopify sneaker stores to see how the average order value of $3145.13 was calculated and why it was "naively" done. 

In [1]:
# importing libraries #

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# importing data #

df = pd.read_csv('data.csv')

In [3]:
# checking to see if data was imported properly #

df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36
1,2,92,925,90,1,cash,2017-03-03 17:38
2,3,44,861,144,1,cash,2017-03-14 4:23
3,4,18,935,156,1,credit_card,2017-03-26 12:43
4,5,18,883,156,1,credit_card,2017-03-01 4:35


In [4]:
# checking for null values #

null_value_bool = df.isnull().values.any()
print("Are there any null values?", null_value_bool)

Are there any null values? False


In [5]:
# checking aggregate values of order_amount and total_items #

agg_list = ["min", "max", "median", "mean", "std"]

df.agg({
    "order_amount" : agg_list,
    "total_items" : agg_list,
    "shop_id" : ["max"]
})

,order_amount,total_items,shop_id
min,90.000000,1.00000,NaN
max,704000.000000,2000.00000,100.0
median,284.000000,2.00000,NaN
mean,3145.128000,8.78720,NaN
std,41282.539349,116.32032,NaN


After loading in the database, a huge disparity was visible between the median and mean of the order amounts... and the standard deviation was ~1300% which is quite high and something that would personally raise red flags. Due to this, the cause of the high standard deviation was investigated.

In [6]:
# checking average sneaker costs #

sneaker_cost_avg = df['order_amount'].sum() / df['total_items'].sum()
print("The average cost of sneakers is:", round(sneaker_cost_avg, 2))

The average cost of sneakers is: 357.92


The average sneaker cost was 357.92 which seems a lot more reasonable (although I have never personally purchased sneakers 100+, it is possible to be this cost as I have witnessed quite expensive ones in store). We explored further to see the large order amounts and numbers.

In [7]:
# checking orders above 1000 order_amount #

bool_mask = (df['order_amount'] > 1000)
df_1000 = df[bool_mask]

df_1000

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00
160,161,78,990,25725,1,credit_card,2017-03-12 5:56
490,491,78,936,51450,2,debit,2017-03-26 17:08
493,494,78,983,51450,2,cash,2017-03-16 21:39
...,...,...,...,...,...,...,...
4646,4647,42,607,704000,2000,credit_card,2017-03-02 4:00
4715,4716,78,818,77175,3,debit,2017-03-05 5:10
4868,4869,42,607,704000,2000,credit_card,2017-03-22 4:00
4882,4883,42,607,704000,2000,credit_card,2017-03-25 4:00


In [8]:
# investigating the high order_amount stores #

expensive_store_array = df_1000['shop_id'].unique()

expensive_store_array

array([42, 78, 43, 54], dtype=int64)

In [9]:
# investigating store 42 #

bool_mask_42 = df_1000['shop_id'] == 42

df_42 = df_1000[bool_mask_42]

df_42.head(10)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00
938,939,42,808,1056,3,credit_card,2017-03-13 23:43
1104,1105,42,607,704000,2000,credit_card,2017-03-24 4:00
1362,1363,42,607,704000,2000,credit_card,2017-03-15 4:00
1364,1365,42,797,1760,5,cash,2017-03-10 6:28
1367,1368,42,926,1408,4,cash,2017-03-13 2:38
1436,1437,42,607,704000,2000,credit_card,2017-03-11 4:00
1471,1472,42,907,1408,4,debit,2017-03-12 23:00


We can see from store 42 that there are repeated order amounts of 704,000 with 2000 total items. This seems like a remarkably large number and is most likely what is causing the high standard deviation. If we look at the other orders, 1056 with 3 items, 1408 with 4, and 1760 with 5, we can see that the sneaker cost is 352 at this store. This also lines up with 704,000 at 2000. However, we see 0 purchases of 2 items so it is possible that there is a reporting error and that whenever a customer purchases 2 of these sneakers at an order amount of 704, it reports 1000x the value. 

In reality, contacting the store to check if the orders are legitimate would be my first course of action but for the sake of this exercise, since there is no public contact information, we "correct" the values.

In [10]:
# investigating store 78 #

bool_mask_78 = df_1000['shop_id'] == 78

df_78 = df_1000[bool_mask_78]

df_78.head(10)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
160,161,78,990,25725,1,credit_card,2017-03-12 5:56
490,491,78,936,51450,2,debit,2017-03-26 17:08
493,494,78,983,51450,2,cash,2017-03-16 21:39
511,512,78,967,51450,2,cash,2017-03-09 7:23
617,618,78,760,51450,2,cash,2017-03-18 11:18
691,692,78,878,154350,6,debit,2017-03-27 22:51
1056,1057,78,800,25725,1,debit,2017-03-15 10:16
1193,1194,78,944,25725,1,debit,2017-03-16 16:38
1204,1205,78,970,25725,1,credit_card,2017-03-17 22:32
1259,1260,78,775,77175,3,credit_card,2017-03-27 9:27


Store 78 has extremely large sneaker values which seem unreasonable to me. According to a light google search, sneaker values can approach 10,000+, however most are in limited quantities and are most likely released online and thus would not be purchasable using cash which seems to be a common method of purchase of these sneakers. Thus, since we can not contact the store, we will remove all records from this store for the calculated metrics. 

In [11]:
# investigating store 43 #

bool_mask_43 = df_1000['shop_id'] == 43

df_43 = df_1000[bool_mask_43]

df_43.head(10)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
3538,3539,43,830,1086,6,debit,2017-03-17 19:56


In [12]:
# investigating store 54 #

bool_mask_54 = df_1000['shop_id'] == 54

df_54 = df_1000[bool_mask_54]

df_54.head(10)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
4141,4142,54,733,1064,8,debit,2017-03-07 17:05


Stores 43 and 54 seem reasonable in price and will not be altered.

Now we remove store 78 from the records, and alter the values of store 42.

In [13]:
# removing store 78 #

bool_mask_78_new = df['shop_id'] != 78

df = df[bool_mask_78_new]

In [14]:
# altering store 42 values #

df.loc[df['order_amount'] == 704000] = 704
df.loc[df['total_items'] == 2000] = 2

Now we've fixed all the values for the database and can see the new and improved metrics.

In [15]:
# checking metrics on revised database #

agg_list = ["min", "max", "median", "mean", "std"]

df.agg({
    "order_amount" : agg_list,
    "total_items" : agg_list,
})

,order_amount,total_items
min,90.000000,1.000000
max,1760.000000,704.000000
median,284.000000,2.000000
mean,303.958014,4.403714
std,162.236359,41.068439


We see a mean of 303.96 which is much better with a standard deviation of 162.24 (or ~50%). The median of 284.00 is also quite close so either metric can be used to evaluate the order_amount.

### Question 2: Why is Adam West working for food suppliers?

#### a) Number of orders from Speedy Express

**SELECT** Shippers.ShipperName **AS** "Shipper", **COUNT**(\*) **AS** "Number_of_Orders"<br>
**FROM** Orders <br>
**JOIN** Shippers <br>
**ON** Shippers.ShipperID = Orders.ShipperID <br>
**WHERE** Shippers.ShipperName = "Speedy Express";

54

#### b) Last name of employee with most orders

The metric used was each individual order counted as 1 order regardless of amount of product in the order

**SELECT** Employees.LastName **AS** "Employee_Surname", **COUNT**(\*) **AS** "Number_of_Orders"<br>
**FROM** Orders<br>
**JOIN** Employees<br>
**ON** Employees.EmployeeID = Orders.EmployeeID<br>
**GROUP BY** Employees.EmployeeID<br>
**ORDER BY** "Number_of_Orders" **DESC**;

Peacock (40 Orders)

#### c) Most ordered product into Germany

The metric used was number of product amongst all orders

**SELECT** Products.ProductName **AS** "Product_Name", **SUM**(OrderDetails.Quantity) **AS** "Total_Quantity"<br>
**FROM** Orders<br>
**JOIN** Customers<br>
**ON** Orders.CustomerID = Customers.CustomerID<br>
**JOIN** OrderDetails<br>
**ON** OrderDetails.OrderID = Orders.OrderID<br>
**JOIN** Products<br>
**ON** Products.ProductID = OrderDetails.ProductID<br>
**WHERE** Customers.Country = "Germany"<br>
**GROUP BY** OrderDetails.ProductID<br>
**ORDER BY** "Total_Quantity" **DESC**;

Boston Crab Meat (160 Orders)

#### Bonus) What's Adam West doing?

It was seen that Adam West of Batman fame was an employee working for the food suppliers so we wanted to see how much he was doing for the company(s).

**SELECT** Employees.LastName **AS** "Employee_Surname", **COUNT**(\*) **AS** "Number_of_Orders"<br>
**FROM** Orders<br>
**JOIN** Employees<br>
**ON** Employees.EmployeeID = Orders.EmployeeID<br>
**GROUP BY** Employees.EmployeeID<br>
**WHERE** Employee_Surname == "West";

No orders showed up. As much as I love Adam West, I recommend an audit into the company that is paying the "honourary" employee who is doing no work. 

### Question 1 Summary

a) Errors in reporting data from the shops

b) Without changing data, median, with changing data, mean still works

c) Mean: 303.96, Median: 284.00